In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from pycaret.classification import *
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
path = '/Users/splaw/Desktop/dyplom/eegParkinsonCognitiveDisorders/EEG'
labels = pd.read_csv("/Users/splaw/Desktop/dyplom/eegParkinsonCognitiveDisorders/dataset.csv", sep=';')

In [8]:
data = []
for filename, label in zip(os.listdir(path), labels['PD']):
    if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf(filepath, preload=True)
        events = mne.make_fixed_length_events(raw, duration=1.0)
        raw.filter(1, 40, fir_design='firwin')
        epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=None)
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=5, fmax=30, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        psds = np.mean(psds, axis=0)
        data.append([psds.tolist(), label])

Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\EEG\PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
1475 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1475 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished


2 bad epochs dropped
Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\EEG\PD130.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 658943  =      0.000 ...  1286.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



...

2 bad epochs dropped
Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\EEG\PD239.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 645119  =      0.000 ...  1259.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1260 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.4s finished


2 bad epochs dropped


In [9]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in data], [x[1] for x in data], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
print(X_test_2d.shape)
print(len(y_test))
print(X_train_2d.shape)
print(len(y_train))
svm = SVC(kernel='linear', C=1, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [139363, 108]

In [5]:
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)